In [237]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import glob
import re
import os
from io import StringIO
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
import PyPDF2

In [4]:
os.chdir('/Users/alisdghnia/Desktop/')

In [5]:
df1 = pd.read_excel('Full Dataset Cleaned with Excel Calculations.xlsx')

In [6]:
df1

,Unnamed: 0,Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Abstract Section,Summary Section,Reference Section,...,Circulating Supply,Abstract Int,Summary Int,Reference Int,Circulating Supply.1,Unnamed: 19,Volume in Currency (24Hr).1,Unnamed: 21,Market Cap.1,Unnamed: 23
0,0,WeTrust1.pdf,1.342260,40,12504,43.02,7.33,True,False,True,...,92148000,1,0,1,92.148M,92148000,3,3.0,228813,228813.0
1,1,Libra4.pdf,0.293671,41,17474,54.42,6.51,True,False,True,...,0,1,0,1,0,0,131,131.0,0,0.0
2,2,B2BX1.pdf,15.404950,42,1,121.22,19.48,False,False,False,...,0,0,0,0,0,0,9220,9220.0,0,0.0
3,3,Bytom1.pdf,0.374002,17,4536,34.15,7.79,True,False,True,...,1764000000,1,0,1,1.764B,1764000000,1.088M,1088000.0,21.831M,21831000.0
4,4,Cindicator1.pdf,1.123809,37,15509,42.92,7.08,False,False,False,...,2000000000,0,0,0,2B,2000000000,65,65.0,1.342M,1342000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,551,CPChain1.pdf,1.195266,34,7203,40.28,6.96,False,False,False,...,376766000,0,0,0,376.766M,376766000,111473,111473.0,1.363M,1363000.0
552,552,Presearch2.pdf,5.291211,39,12904,41.70,7.16,True,False,False,...,396578000,1,0,0,396.578M,396578000,107073,107073.0,33.127M,33127000.0
553,553,Electra1.pdf,0.624846,34,5508,37.13,9.05,True,True,False,...,0,1,1,0,0,0,48,48.0,0,0.0
554,554,Blockmason1.pdf,3.355893,31,13640,49.04,7.00,False,False,False,...,116159000,0,0,0,116.159M,116159000,9,9.0,283474,283474.0


In [7]:
df2 = pd.read_excel('TFIDF smaller Dataset.xlsx')

In [8]:
df2

,File Name,aarhus,abbreviate,abdelmalek,abound,abstracts,abu,accelerates,accessories,acclaim,...,Ô¨Åfth,Ô¨Ånalised,Ô¨Ånalization,Ô¨Åred,Ô¨Årmly,Ô¨Åscal,Ô¨Åxedpoint,Ô¨Çagged,Ô¨Çoat,Ô¨Çuid
0,WeTrust1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Libra4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B2BX1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bytom1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cindicator1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,CPChain1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
550,Presearch2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
551,Electra1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552,Blockmason1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df3 = pd.read_csv('Files Texts Dataframe.csv')

In [10]:
df3 = df3.rename(columns= {'Unnamed: 0':'File Name'})

In [11]:
df3

,File Name,Text Corpus
0,WeTrust1,wetrust whitepaper table of contents \n\n \n\n...
1,Libra4,state machine replication in the libra\n\nbloc...
2,B2BX1,23\n\n
3,Bytom1,bytomv1.0 ...
4,Cindicator1,hybrid intelligence for\n\neective asset mana...
...,...,...
551,CPChain1,\t\n\ncyber-physical chain (cpchain) whitepape...
552,Presearch2,whitepaper\n\nthe community-powered search eng...
553,Electra1,"white paper\n\nrelease 1.0 / january 31, 2018\..."
554,Blockmason1,the credit protocol whitepaper v1.0.1\n\nblock...


In [12]:
tfidf = pd.read_csv('TFIDF smaller Dataset.csv')

In [13]:
tfidf

,File Name,aarhus,abbreviate,abdelmalek,abound,abstracts,abu,accelerates,accessories,acclaim,...,ﬁfth,ﬁnalised,ﬁnalization,ﬁred,ﬁrmly,ﬁscal,ﬁxedpoint,ﬂagged,ﬂoat,ﬂuid
0,WeTrust1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Libra4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B2BX1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bytom1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cindicator1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,CPChain1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
550,Presearch2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
551,Electra1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552,Blockmason1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(tfidf_array, price_array, 
                                                    random_state = 0)

#count_vect = CountVectorizer(stop_words= 'english',lowercase=True)
#X_train_counts = count_vect.fit_transform(X_train)

#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train, y_train)

ValueError: Unknown label type: (array([4.018700e+00, 3.416010e-01, 5.840000e-04, 2.976000e-03,
       1.572000e-03, 9.160000e-04, 3.727100e-02, 8.495700e+00,
       6.000000e-05, 1.212000e+00, 5.315000e-03, 4.273600e-02,
       1.160000e-04, 4.541000e-03, 8.470000e-04, 1.100000e-05,
       1.573400e+02, 9.704000e+01, 2.704000e-02, 1.173400e+00,
       1.547000e+01, 4.541000e-03, 4.740480e-01, 7.345200e-02,
       1.170000e-04, 1.573400e+02, 1.529450e-01, 7.751000e-03,
       4.682000e-03, 7.870000e-04, 1.820000e-04, 3.174120e-01,
       1.126100e-02, 3.462900e-02, 3.685500e-01, 4.325200e-02,
       1.000200e+00, 8.061120e-01, 5.364700e-02, 9.800000e-07,
       1.969900e-02, 1.573400e+02, 2.253100e-02, 1.170000e-04,
       5.665000e-03, 1.014200e+00, 3.619000e-03, 1.221000e-01,
       1.732200e-02, 3.090420e-01, 6.420000e-04, 0.000000e+00,
       1.478000e-03, 8.061120e-01, 3.685500e-01, 1.116000e-03,
       3.868600e-02, 1.170000e-04, 5.205000e+01, 3.416010e-01,
       8.353100e-02, 2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['Text Corpus'], df['Price (Intraday)'], 
                                                    random_state = 0)

count_vect = CountVectorizer(stop_words= 'english',lowercase=True)
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

ValueError: Unknown label type: (array([3.732000e-03, 8.061120e-01, 5.840000e-04, 2.976000e-03,
       1.572000e-03, 3.700000e-04, 3.727100e-02, 1.682820e+03,
       1.437170e-01, 1.534020e-01, 9.823000e-02, 6.420000e-04,
       2.034000e+01, 9.888000e-03, 4.164000e-02, 6.000000e-05,
       1.573400e+02, 9.704000e+01, 2.704000e-02, 4.325200e-02,
       3.084800e-02, 6.272430e-01, 1.140000e-03, 7.345200e-02,
       1.170000e-04, 1.573400e+02, 1.871400e-02, 7.751000e-03,
       4.682000e-03, 7.870000e-04, 1.130000e-06, 2.231400e-02,
       4.541000e-03, 3.462900e-02, 7.629000e+01, 1.573400e+02,
       1.000200e+00, 2.114200e+00, 5.364700e-02, 2.778650e-01,
       1.026400e+00, 1.259000e-02, 4.740480e-01, 1.170000e-04,
       5.665000e-03, 1.014200e+00, 3.619000e-03, 8.495700e+00,
       1.732200e-02, 2.277800e-02, 1.547000e+01, 3.667000e+00,
       3.685500e-01, 8.110000e-06, 3.987000e-03, 4.119770e-01,
       3.621000e-02, 1.170000e-04, 2.920000e-04, 8.061120e-01,
       8.353100e-02, 2

In [14]:
df3['category_id'] = df3['Text Corpus'].factorize()[0]

In [15]:
from io import StringIO
category_id_df = df3[['File Name', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'File Name']].values)
df3

,File Name,Text Corpus,category_id
0,WeTrust1,wetrust whitepaper table of contents \n\n \n\n...,0
1,Libra4,state machine replication in the libra\n\nbloc...,1
2,B2BX1,23\n\n,2
3,Bytom1,bytomv1.0 ...,3
4,Cindicator1,hybrid intelligence for\n\neective asset mana...,4
...,...,...,...
551,CPChain1,\t\n\ncyber-physical chain (cpchain) whitepape...,539
552,Presearch2,whitepaper\n\nthe community-powered search eng...,540
553,Electra1,"white paper\n\nrelease 1.0 / january 31, 2018\...",541
554,Blockmason1,the credit protocol whitepaper v1.0.1\n\nblock...,542


In [16]:
import re
import os
from io import StringIO
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', str(text))
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace


def remove_stops(text, stops):
    text = re.sub(r"AC\/\d{1,4}\/\d{1,4}", "", str(text))
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

In [17]:
new_corpus = []
for i in df3['Text Corpus']:
    i = clean_text(i)
    i = remove_stops(i, 'english')
    new_corpus.append(i)

df3['New Corpus'] = new_corpus

In [18]:
df3

,File Name,Text Corpus,category_id,New Corpus
0,WeTrust1,wetrust whitepaper table of contents \n\n \n\n...,0,wetrust whitepaper table of contents abstract ...
1,Libra4,state machine replication in the libra\n\nbloc...,1,state machine replication in the libra blockch...
2,B2BX1,23\n\n,2,
3,Bytom1,bytomv1.0 ...,3,bytomv bytom an interoperation protocol for di...
4,Cindicator1,hybrid intelligence for\n\neective asset mana...,4,hybrid intelligence for eective asset managem...
...,...,...,...,...
551,CPChain1,\t\n\ncyber-physical chain (cpchain) whitepape...,539,cyberphysical chain cpchain whitepaper decentr...
552,Presearch2,whitepaper\n\nthe community-powered search eng...,540,whitepaper the communitypowered search engine ...
553,Electra1,"white paper\n\nrelease 1.0 / january 31, 2018\...",541,white paper release january table of contents ...
554,Blockmason1,the credit protocol whitepaper v1.0.1\n\nblock...,542,the credit protocol whitepaper v blockmason te...


In [19]:
df3 = df3.drop(columns=['Text Corpus'])

In [20]:
df3

,File Name,category_id,New Corpus
0,WeTrust1,0,wetrust whitepaper table of contents abstract ...
1,Libra4,1,state machine replication in the libra blockch...
2,B2BX1,2,
3,Bytom1,3,bytomv bytom an interoperation protocol for di...
4,Cindicator1,4,hybrid intelligence for eective asset managem...
...,...,...,...
551,CPChain1,539,cyberphysical chain cpchain whitepaper decentr...
552,Presearch2,540,whitepaper the communitypowered search engine ...
553,Electra1,541,white paper release january table of contents ...
554,Blockmason1,542,the credit protocol whitepaper v blockmason te...


In [21]:
df1['Name'] = df1['Name'].apply(lambda x: x.replace('.pdf', ''))

In [22]:
df1

,Unnamed: 0,Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Abstract Section,Summary Section,Reference Section,...,Circulating Supply,Abstract Int,Summary Int,Reference Int,Circulating Supply.1,Unnamed: 19,Volume in Currency (24Hr).1,Unnamed: 21,Market Cap.1,Unnamed: 23
0,0,WeTrust1,1.342260,40,12504,43.02,7.33,True,False,True,...,92148000,1,0,1,92.148M,92148000,3,3.0,228813,228813.0
1,1,Libra4,0.293671,41,17474,54.42,6.51,True,False,True,...,0,1,0,1,0,0,131,131.0,0,0.0
2,2,B2BX1,15.404950,42,1,121.22,19.48,False,False,False,...,0,0,0,0,0,0,9220,9220.0,0,0.0
3,3,Bytom1,0.374002,17,4536,34.15,7.79,True,False,True,...,1764000000,1,0,1,1.764B,1764000000,1.088M,1088000.0,21.831M,21831000.0
4,4,Cindicator1,1.123809,37,15509,42.92,7.08,False,False,False,...,2000000000,0,0,0,2B,2000000000,65,65.0,1.342M,1342000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,551,CPChain1,1.195266,34,7203,40.28,6.96,False,False,False,...,376766000,0,0,0,376.766M,376766000,111473,111473.0,1.363M,1363000.0
552,552,Presearch2,5.291211,39,12904,41.70,7.16,True,False,False,...,396578000,1,0,0,396.578M,396578000,107073,107073.0,33.127M,33127000.0
553,553,Electra1,0.624846,34,5508,37.13,9.05,True,True,False,...,0,1,1,0,0,0,48,48.0,0,0.0
554,554,Blockmason1,3.355893,31,13640,49.04,7.00,False,False,False,...,116159000,0,0,0,116.159M,116159000,9,9.0,283474,283474.0


In [23]:
df = pd.merge(df1, df3, left_on = 'Name', right_on= 'File Name')

In [24]:
df

,Unnamed: 0,Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Abstract Section,Summary Section,Reference Section,...,Reference Int,Circulating Supply.1,Unnamed: 19,Volume in Currency (24Hr).1,Unnamed: 21,Market Cap.1,Unnamed: 23,File Name,category_id,New Corpus
0,0,WeTrust1,1.342260,40,12504,43.02,7.33,True,False,True,...,1,92.148M,92148000,3,3.0,228813,228813.0,WeTrust1,0,wetrust whitepaper table of contents abstract ...
1,1,Libra4,0.293671,41,17474,54.42,6.51,True,False,True,...,1,0,0,131,131.0,0,0.0,Libra4,1,state machine replication in the libra blockch...
2,2,B2BX1,15.404950,42,1,121.22,19.48,False,False,False,...,0,0,0,9220,9220.0,0,0.0,B2BX1,2,
3,3,Bytom1,0.374002,17,4536,34.15,7.79,True,False,True,...,1,1.764B,1764000000,1.088M,1088000.0,21.831M,21831000.0,Bytom1,3,bytomv bytom an interoperation protocol for di...
4,4,Cindicator1,1.123809,37,15509,42.92,7.08,False,False,False,...,0,2B,2000000000,65,65.0,1.342M,1342000.0,Cindicator1,4,hybrid intelligence for eective asset managem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,551,CPChain1,1.195266,34,7203,40.28,6.96,False,False,False,...,0,376.766M,376766000,111473,111473.0,1.363M,1363000.0,CPChain1,539,cyberphysical chain cpchain whitepaper decentr...
552,552,Presearch2,5.291211,39,12904,41.70,7.16,True,False,False,...,0,396.578M,396578000,107073,107073.0,33.127M,33127000.0,Presearch2,540,whitepaper the communitypowered search engine ...
553,553,Electra1,0.624846,34,5508,37.13,9.05,True,True,False,...,0,0,0,48,48.0,0,0.0,Electra1,541,white paper release january table of contents ...
554,554,Blockmason1,3.355893,31,13640,49.04,7.00,False,False,False,...,0,116.159M,116159000,9,9.0,283474,283474.0,Blockmason1,542,the credit protocol whitepaper v blockmason te...


In [25]:
df.columns

Index(['Unnamed: 0', 'Name', 'File Size (MB)', 'Number of Pages', 'Word Count',
       'Flesch Reading Ease', 'New Dale-Chall', 'Abstract Section',
       'Summary Section', 'Reference Section', 'Crypto Name',
       'Price (Intraday)', 'Market Cap', 'Volume in Currency (24Hr)',
       'Circulating Supply', 'Abstract Int', 'Summary Int', 'Reference Int',
       'Circulating Supply.1', 'Unnamed: 19', 'Volume in Currency (24Hr).1',
       'Unnamed: 21', 'Market Cap.1', 'Unnamed: 23', 'File Name',
       'category_id', 'New Corpus'],
      dtype='object')

In [26]:
df = df.drop(columns= ['Unnamed: 0', 'Unnamed: 19', 'Unnamed: 21','Unnamed: 23', 'Circulating Supply.1', 'Market Cap.1', 'Volume in Currency (24Hr).1', 'Circulating Supply',
            'Summary Section', 'Reference Section', 'Abstract Section'])

In [27]:
df.columns

Index(['Name', 'File Size (MB)', 'Number of Pages', 'Word Count',
       'Flesch Reading Ease', 'New Dale-Chall', 'Crypto Name',
       'Price (Intraday)', 'Market Cap', 'Volume in Currency (24Hr)',
       'Abstract Int', 'Summary Int', 'Reference Int', 'File Name',
       'category_id', 'New Corpus'],
      dtype='object')

In [28]:
df

,Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Crypto Name,Price (Intraday),Market Cap,Volume in Currency (24Hr),Abstract Int,Summary Int,Reference Int,File Name,category_id,New Corpus
0,WeTrust1,1.342260,40,12504,43.02,7.33,WeTrust USD,2.483000e-03,228813.0,3.0,1,0,1,WeTrust1,0,wetrust whitepaper table of contents abstract ...
1,Libra4,0.293671,41,17474,54.42,6.51,Libera USD,4.740480e-01,0.0,131.0,1,0,1,Libra4,1,state machine replication in the libra blockch...
2,B2BX1,15.404950,42,1,121.22,19.48,X USD,6.870000e-04,0.0,9220.0,0,0,0,B2BX1,2,
3,Bytom1,0.374002,17,4536,34.15,7.79,Bytom USD,1.237500e-02,21831000.0,1088000.0,1,0,1,Bytom1,3,bytomv bytom an interoperation protocol for di...
4,Cindicator1,1.123809,37,15509,42.92,7.08,Cindicator USD,6.710000e-04,1342000.0,65.0,0,0,0,Cindicator1,4,hybrid intelligence for eective asset managem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,CPChain1,1.195266,34,7203,40.28,6.96,CPChain USD,3.619000e-03,1363000.0,111473.0,0,0,0,CPChain1,539,cyberphysical chain cpchain whitepaper decentr...
552,Presearch2,5.291211,39,12904,41.70,7.16,Presearch USD,8.353100e-02,33127000.0,107073.0,1,0,0,Presearch2,540,whitepaper the communitypowered search engine ...
553,Electra1,0.624846,34,5508,37.13,9.05,Electra USD,4.600000e-07,0.0,48.0,1,1,0,Electra1,541,white paper release january table of contents ...
554,Blockmason1,3.355893,31,13640,49.04,7.00,Blockmason Credit Protocol USD,2.440000e-03,283474.0,9.0,0,0,0,Blockmason1,542,the credit protocol whitepaper v blockmason te...


In [29]:
df.describe()

,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Price (Intraday),Market Cap,Volume in Currency (24Hr),Abstract Int,Summary Int,Reference Int,category_id
count,556.000000,556.000000,556.000000,556.000000,556.00000,556.000000,5.560000e+02,5.560000e+02,556.000000,556.000000,556.000000,556.000000
mean,2.404612,29.823741,9039.618705,-153.014568,7.90295,189.292141,3.356313e+09,3.121680e+08,0.429856,0.212230,0.383094,269.147482
std,3.921081,38.484740,17582.547188,4386.642714,4.47068,1944.097643,3.033648e+10,2.797582e+09,0.495501,0.409255,0.486579,157.979380
min,0.038481,1.000000,0.000000,-103290.780000,0.00000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,-1.000000
25%,0.477026,15.000000,4153.000000,40.480000,7.01750,0.001606,8.831550e+04,3.640000e+02,0.000000,0.000000,0.000000,132.750000
50%,1.131416,24.000000,6671.500000,46.780000,7.44000,0.023677,2.149000e+06,8.006500e+04,0.000000,0.000000,0.000000,267.500000
75%,2.567190,37.000000,11000.000000,52.925000,7.89500,0.341601,8.369150e+07,4.912500e+06,1.000000,0.000000,1.000000,406.250000
max,32.407803,684.000000,332886.000000,206.840000,82.78000,22908.160000,4.379420e+11,4.785800e+10,1.000000,1.000000,1.000000,543.000000


In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, max_df=0.01, min_df=5, norm='l2', lowercase= True,
                        encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df['New Corpus']).toarray()
labels = df.category_id
features.shape

(556, 12031)

In [203]:
max(features[3])

0.2764754995025772

In [181]:
features, labels

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 0        0
 1        1
 2        2
 3        3
 4        4
       ... 
 551    539
 552    540
 553    541
 554    542
 555    543
 Name: category_id, Length: 556, dtype: int64)

In [73]:
df['tfidf'] = list(features)

In [74]:
df

,Name,File Size (MB),Number of Pages,Word Count,Flesch Reading Ease,New Dale-Chall,Crypto Name,Price (Intraday),Market Cap,Volume in Currency (24Hr),Abstract Int,Summary Int,Reference Int,File Name,category_id,New Corpus,tfidf
0,WeTrust1,1.342260,40,12504,43.02,7.33,WeTrust USD,2.483000e-03,228813.0,3.0,1,0,1,WeTrust1,0,wetrust whitepaper table of contents abstract ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.06372149602432792,..."
1,Libra4,0.293671,41,17474,54.42,6.51,Libera USD,4.740480e-01,0.0,131.0,1,0,1,Libra4,1,state machine replication in the libra blockch...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,B2BX1,15.404950,42,1,121.22,19.48,X USD,6.870000e-04,0.0,9220.0,0,0,0,B2BX1,2,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Bytom1,0.374002,17,4536,34.15,7.79,Bytom USD,1.237500e-02,21831000.0,1088000.0,1,0,1,Bytom1,3,bytomv bytom an interoperation protocol for di...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Cindicator1,1.123809,37,15509,42.92,7.08,Cindicator USD,6.710000e-04,1342000.0,65.0,0,0,0,Cindicator1,4,hybrid intelligence for eective asset managem...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,CPChain1,1.195266,34,7203,40.28,6.96,CPChain USD,3.619000e-03,1363000.0,111473.0,0,0,0,CPChain1,539,cyberphysical chain cpchain whitepaper decentr...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
552,Presearch2,5.291211,39,12904,41.70,7.16,Presearch USD,8.353100e-02,33127000.0,107073.0,1,0,0,Presearch2,540,whitepaper the communitypowered search engine ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
553,Electra1,0.624846,34,5508,37.13,9.05,Electra USD,4.600000e-07,0.0,48.0,1,1,0,Electra1,541,white paper release january table of contents ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
554,Blockmason1,3.355893,31,13640,49.04,7.00,Blockmason Credit Protocol USD,2.440000e-03,283474.0,9.0,0,0,0,Blockmason1,542,the credit protocol whitepaper v blockmason te...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


from sklearn.feature_selection import chi2
import numpy as np
N = 2
for file_name, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(file_name))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

In [30]:
X = df['New Corpus']
y = df['Price (Intraday)']
#y = df['Abstract Int']

In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [164]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

knn_clf = Pipeline([('tfidf', TfidfVectorizer(min_df = 5)),
                    ('clf', KNeighborsClassifier(n_neighbors= 5))])
            
knn_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5)),
                ('clf', KNeighborsClassifier())])

In [165]:
pred = knn_clf.predict(X_test)

In [166]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, pred))

[[104  21]
 [ 52  46]]


In [167]:
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.67      0.83      0.74       125
           1       0.69      0.47      0.56        98

    accuracy                           0.67       223
   macro avg       0.68      0.65      0.65       223
weighted avg       0.68      0.67      0.66       223



In [168]:
pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1])

In [133]:
X = df[['File Size (MB)', 'Number of Pages', 'Word Count']]

In [135]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [136]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()

dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [137]:
X_test, y_test

(     File Size (MB)  Number of Pages  Word Count
 457        2.626913               23       14990
 167        0.960614               23        4496
 55         4.128945               79       19464
 245        0.172790               27        5472
 70         1.148368               30       10655
 ..              ...              ...         ...
 320        1.565772               34       20282
 331        1.393498               25        1223
 324        0.145105                8        4144
 16         0.205909               13        3794
 48         0.662073               37       26475
 
 [223 rows x 3 columns],
 457    1
 167    0
 55     0
 245    0
 70     0
       ..
 320    0
 331    0
 324    1
 16     0
 48     1
 Name: Abstract Int, Length: 223, dtype: int64)

In [139]:
X_array = np.array(X_test)
X_array

array([[2.62691307e+00, 2.30000000e+01, 1.49900000e+04],
       [9.60614204e-01, 2.30000000e+01, 4.49600000e+03],
       [4.12894535e+00, 7.90000000e+01, 1.94640000e+04],
       [1.72789574e-01, 2.70000000e+01, 5.47200000e+03],
       [1.14836788e+00, 3.00000000e+01, 1.06550000e+04],
       [1.14335060e-01, 8.00000000e+00, 1.15500000e+03],
       [5.96781731e-01, 2.60000000e+01, 6.28300000e+03],
       [5.07930279e+00, 6.20000000e+01, 1.57620000e+04],
       [3.58468246e+00, 2.70000000e+01, 1.07640000e+04],
       [3.51651669e+00, 3.40000000e+01, 8.18700000e+03],
       [8.11580658e-01, 2.00000000e+01, 4.59800000e+03],
       [1.95852280e-01, 1.30000000e+01, 5.70200000e+03],
       [1.33842640e+01, 2.90000000e+01, 1.17450000e+04],
       [1.68009758e-01, 8.00000000e+00, 3.18200000e+03],
       [1.37470913e+00, 3.50000000e+01, 9.77300000e+03],
       [5.94076157e-01, 1.80000000e+01, 3.16000000e+03],
       [3.09043226e+01, 1.80000000e+01, 2.16100000e+03],
       [1.14235115e+00, 4.20000

In [140]:
pred2 = dt_clf.predict(X_array)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [144]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, pred2))

[[79 46]
 [40 58]]


In [153]:
print(metrics.classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.66      0.63      0.65       125
           1       0.56      0.59      0.57        98

    accuracy                           0.61       223
   macro avg       0.61      0.61      0.61       223
weighted avg       0.62      0.61      0.62       223



In [141]:
pred2

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1])

In [ ]:
from sklearn import tree
tree.plot_tree(dt_clf)

In [185]:
X = df['New Corpus']
#y = all_df['Price (Intraday)']
y = df['Abstract Int']

In [191]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [171]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

sgd_clf = Pipeline([('tfidf', TfidfVectorizer(min_df = 5)),
                    ('clf', SGDClassifier(loss="hinge", penalty="l2", max_iter=5))])
            
sgd_clf.fit(X_train, y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5)),
                ('clf', SGDClassifier(max_iter=5))])

In [172]:
pred3 = sgd_clf.predict(X_test)

In [173]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, pred3))

[[104  21]
 [ 44  54]]


In [174]:
print(metrics.classification_report(y_test, pred3))

              precision    recall  f1-score   support

           0       0.70      0.83      0.76       125
           1       0.72      0.55      0.62        98

    accuracy                           0.71       223
   macro avg       0.71      0.69      0.69       223
weighted avg       0.71      0.71      0.70       223



In [175]:
pred3

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1])

In [187]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

mlp_clf = Pipeline([('tfidf', TfidfVectorizer(min_df = 5)),
                    ('clf', MLPClassifier(max_iter=5))])
                    
mlp_clf.fit(X_train, y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5)),
                ('clf', MLPClassifier(max_iter=5))])

In [188]:
pred4 = mlp_clf.predict(X_test)

In [189]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, pred4))

[[102  23]
 [ 58  40]]


In [179]:
print(metrics.classification_report(y_test, pred4))

              precision    recall  f1-score   support

           0       0.64      0.92      0.75       125
           1       0.77      0.34      0.47        98

    accuracy                           0.66       223
   macro avg       0.70      0.63      0.61       223
weighted avg       0.70      0.66      0.63       223



In [180]:
pred4

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1])

In [190]:
X = df['New Corpus']
y = df['Price (Intraday)']
#y = df['Abstract Int']

In [192]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

mlp_clf = Pipeline([('tfidf', TfidfVectorizer(min_df = 5)),
                    ('clf', MLPRegressor())])
            
mlp_clf.fit(X_train, y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5)), ('clf', MLPRegressor())])

In [193]:
pred5 = mlp_clf.predict(X_test)

In [194]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, pred5))

ValueError: continuous is not supported

In [195]:
pred5

array([ 29.5443538 , -17.58339306,   5.85727919,  54.31655871,
        30.46618597, -13.41382097,   1.56407838,  -3.99266448,
        20.25711729,   7.00211708,  12.53390926,  -1.36764828,
        28.12681426,  37.17612523,  27.82737194,  60.34786601,
        22.99548731,  37.8321521 ,  69.06733541,  19.72258116,
        -0.38002663,  28.37321505,  30.24457568,   1.47594849,
         5.34282583,   3.12503295, -14.679577  ,  34.76172729,
        11.40621631,  -2.95871889,  34.34978941,  -8.66336718,
        12.30939907,  13.79273914,  11.40621631,   7.34284741,
        43.28179662,  11.40621631,  31.56915823,  26.43451892,
        14.8153136 , -25.59633642, -35.61900851,  21.39116976,
        11.40621631,  38.70782793,   2.67398346,  43.49071117,
        23.69099284,  22.04067422,  18.73473462,   8.01841809,
       -13.82893026,   4.66957412,  38.62139676,  -6.61645874,
        44.30553365,   9.95870329,  18.90739664,  29.45275088,
        -3.65253167,  18.33422888,  65.14307483,  32.40

In [198]:
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
y_train_arr = np.array(y_train)

In [37]:
y_arr = y_arr.reshape(-1,1)

In [73]:
from sklearn.preprocessing import MinMaxScaler
y_train_prep = MinMaxScaler(y_train)

In [74]:
y_train_array = np.array(y_train_prep)

In [75]:
y_train_array

array(MinMaxScaler(feature_range=231    0.003037
420    0.001969
3      0.012375
280    0.001145
429    5.892000
         ...   
71     0.311247
106    2.279800
270    0.000244
435    8.671500
102    0.143717
Name: Price (Intraday), Length: 333, dtype: float64), dtype=object)

In [85]:
y_train_prep

MinMaxScaler(feature_range=231    0.003037
420    0.001969
3      0.012375
280    0.001145
429    5.892000
         ...   
71     0.311247
106    2.279800
270    0.000244
435    8.671500
102    0.143717
Name: Price (Intraday), Length: 333, dtype: float64)

In [64]:
X = df['New Corpus']
y = df['Price (Intraday)']
#y = df['Abstract Int']

In [100]:
X = df[['File Size (MB)', 'Number of Pages', 'Word Count']]

In [102]:
X_train

,File Size (MB),Number of Pages,Word Count
231,2.565236,57,15255
420,7.036593,45,9423
3,0.374002,17,4536
280,0.063487,5,1400
429,2.012864,41,8327
...,...,...,...
71,0.375694,88,31688
106,5.203882,35,30
270,9.094313,71,18206
435,3.887333,21,20059


In [101]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [87]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

mlp_clf = Pipeline([('tfidf', TfidfVectorizer(min_df = 5)),
                    ('clf', MLPRegressor())])
            
model = mlp_clf.fit(X_train, y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [91]:
model.predict(X_test)

array([ 37.57612052,  -2.83277991,  18.58443461,  57.39604034,
        34.28606744,  -4.4133646 ,   9.6923705 ,   4.48171161,
        27.3963168 ,  14.77675889,  22.70526846,  16.01028086,
        37.390628  ,  41.52140611,  36.06271613,  60.704513  ,
        33.17503667,  43.94983662,  69.05677258,  26.8108496 ,
        13.98795741,  32.94572171,  35.56219451,  14.21536513,
        18.38287409,  10.50132206,  -1.62005107,  41.63365038,
         9.99078113,   8.59710474,  37.85509404,   4.40289028,
        22.63972876,  22.88852386,   9.99078113,  11.15759371,
        49.96744201,   9.99078113,  37.43486508,  32.84257244,
        19.81788803, -11.04207641, -16.430305  ,  36.04136299,
         9.99078113,  42.55786693,  15.75680551,  45.1369678 ,
        29.17119824,  31.65679348,  19.51041179,  17.29078186,
        -1.06208816,   8.32893388,  39.88730937,  -2.67380013,
        48.12312754,  21.10942465,  30.51744697,  35.86391134,
         7.30126498,  30.30262042,  65.05323929,  41.07

In [92]:
model.score(X_test, y_test)

-0.010142836654473797

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor

sgd_clf = Pipeline([('tfidf', TfidfVectorizer(min_df = 5)),
                    ('clf', SGDRegressor(penalty="l2", max_iter=5))])
            
sgd_clf.fit(X_train, y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5)),
                ('clf', SGDRegressor(max_iter=5))])

In [98]:
sgd_clf.predict(X_test)

array([9.63005634e+01, 7.28555471e+01, 9.95639350e+01, 1.01331732e+02,
       8.13826537e+01, 1.43622457e+00, 8.20304250e+01, 8.51153672e+01,
       9.82433468e+01, 9.30238763e+01, 8.46070954e+01, 8.95022032e+01,
       9.79096991e+01, 7.93841709e+01, 9.99619653e+01, 7.77328695e+01,
       8.34666517e+01, 9.55139357e+01, 9.73684067e+01, 8.06627555e+01,
       8.26492198e+01, 9.16417356e+01, 1.00922232e+02, 8.25558972e+01,
       8.36114667e+01, 6.09436005e+01, 7.73129888e+01, 9.68752036e+01,
       1.47565344e+00, 8.30548891e+01, 9.13960722e+01, 8.92185137e+01,
       7.22753508e+01, 1.08884324e+02, 1.47565344e+00, 7.58938811e+01,
       9.60523222e+01, 1.47565344e+00, 8.89596441e+01, 8.79035682e+01,
       9.16815669e+01, 7.09391789e+01, 7.44737791e+01, 9.28758436e+01,
       1.47565344e+00, 9.15129399e+01, 7.42062186e+01, 8.86022522e+01,
       8.30495509e+01, 9.53661468e+01, 2.34254735e+01, 8.25561965e+01,
       7.38574825e+01, 4.33839296e+01, 5.91230125e+01, 5.59386903e+01,
      

In [99]:
sgd_clf.score(X_test, y_test)

-0.007647019349140027

In [103]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor

sgd_regressor = SGDRegressor(penalty="l2", max_iter=5)
            
sgd_regressor.fit(X_train, y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(max_iter=5)

In [105]:
sgd_regressor.score(X_test, y_test)

-2.0370506114626504e+26

In [106]:
from sklearn.metrics import confusion_matrix

In [192]:
y = df['Market Cap']

In [193]:
X = df['Number of Pages']

In [226]:
from sklearn.preprocessing import MinMaxScaler
y_train_prep = MinMaxScaler(y)

In [227]:
y_train_prep

MinMaxScaler(feature_range=0        228813.0
1             0.0
2             0.0
3      21831000.0
4       1342000.0
          ...    
551     1363000.0
552    33127000.0
553           0.0
554      283474.0
555     1037000.0
Name: Market Cap, Length: 556, dtype: float64)

In [216]:
y_train_array = np.array(y_train_prep)

In [240]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [242]:
X_train = np.array(X_train)

In [243]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

mlp_regressor = MLPRegressor(max_iter=5)
            
mlp_regressor.fit(X_train.reshape(-1,1), y_train)

/Users/alisdghnia/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=5)

In [232]:
X_test = np.array(X_test)

In [233]:
mlp_regressor.score(X_test.reshape(-1,1), y_test)

-0.013569240780046732

In [234]:
mlp_regressor.predict(X_test.reshape(-1,1))

array([ 7.50412104e+00,  3.40739219e+00,  8.32376796e+00,  1.77036054e+00,
        2.79452666e+00,  1.26269143e+01, -1.21123761e-01,  4.63535683e+00,
        1.77036054e+00,  3.20310368e+00,  2.59023815e+00,  1.16023556e+01,
        4.02062164e+00,  1.05777970e+01,  5.25009202e+00,  9.96306179e+00,
        9.96306179e+00,  4.63535683e+00,  3.40739219e+00,  6.88938585e+00,
        4.43044510e+00,  2.30774125e+01,  7.28749563e-01,  1.77497075e+01,
        1.28318260e+01,  1.15031089e+00,  8.73359142e+00,  2.79452666e+00,
        9.34832660e+00,  5.65991548e+00,  4.22553337e+00,  2.18166113e+00,
        1.24220025e+01,  7.29920931e+00,  1.56461624e+00,  5.65991548e+00,
        5.04518029e+00,  1.28318260e+01,  5.25009202e+00,  1.30367377e+01,
        8.11885623e+00,  1.46760316e+01,  4.02062164e+00,  3.61168070e+00,
        1.13974439e+01,  4.63535683e+00,  3.09107502e-01, -1.21123761e-01,
        1.15031089e+00,  9.34832660e+00,  5.25009202e+00,  9.14341488e+00,
        6.47956239e+00,  

In [235]:
mlp_regressor.score(X_test.reshape(-1,1), y_test)

-0.013569240780046732

In [223]:
y_train_prep

MinMaxScaler(feature_range=0        228813.0
1             0.0
2             0.0
3      21831000.0
4       1342000.0
          ...    
551     1363000.0
552    33127000.0
553           0.0
554      283474.0
555     1037000.0
Name: Market Cap, Length: 556, dtype: float64)

In [238]:
y

0        228813.0
1             0.0
2             0.0
3      21831000.0
4       1342000.0
          ...    
551     1363000.0
552    33127000.0
553           0.0
554      283474.0
555     1037000.0
Name: Market Cap, Length: 556, dtype: float64